In [1]:
from child import Child
import helper_functions
import sandia_stats
import time

In [2]:
# Step 0: Read config file
with open('phasing_config_file.txt', 'r') as f:
    config_elem = {}
    for line in f:
        line_split = line.strip().split('\t')
        #print(line_split)
        config_elem[line_split[0]] = line_split[1]

In [3]:
# Step 1: Obtain names from ped file
names = helper_functions.ped_file_reader(config_elem["PED_FILE"])

In [4]:
# Testing: print names
#print(names)

In [5]:
# Step 2: Read in the vcf file
df = helper_functions.read_VCF(config_elem["VCF_FILE"], names)

C:\Users\rodriguezah\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (0,5) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
# for testing: print the header, then the footer
#df.head()
#df.tail()

In [7]:
# Step 3: obtain only high quality SNPs
SNP_df = helper_functions.SNP_filter(df)

In [8]:
# testing purposes
#SNP_df.head()

In [9]:
# Step 4: creat chromosome list
chromosome_list = [i for i in range(1,23)]

In [10]:
# testing purposes
#print(chromosome_list)

In [11]:
# Step 5: Create child object for proband
proband = Child(config_elem["PROBAND_NAME"], names[0], names[1], SNP_df)

In [12]:
def sandia_t_test_snps(vcf_pos, maternal_rd, paternal_rd, samp_size=10000, t_thres=25):
    mom_minus_samp_size = maternal_rd.size - samp_size
    if 0 > mom_minus_samp_size:
        print("Sample size is larger than total number of data points")
        return None
    else:
        # Step 1: Allocate an array that will store the t
        t_values = []
        # Step 2: Generate difference array between mom and dad
        diff_arr = (maternal_rd - paternal_rd).tolist()
        # Step 3: Calculate initial moments
        moments = sandia_stats.m1_m2_moment_generator(diff_arr[0:samp_size])
        # Step 4: Calculate t-statistic for first window
        t_values.append(abs(moments[0] / (moments[1]**0.5/samp_size)))
        # Step 5: Calculate t-values for rest of positions
        counter2 = samp_size
        mom_update_func = sandia_stats.m1_m2_moment_updater
        for i in range(mom_minus_samp_size):
            moments = mom_update_func(moments, diff_arr[i], diff_arr[counter2], samp_size)
            counter2 += 1
            t_values.append(abs(moments[0] / (moments[1]**0.5/samp_size)))
        # Step 6: See if any t-values exceed the t-value threshold
        index_of_mosaicism = next((i for i, elem in enumerate(t_values) if elem > t_thres), -1)
        if index_of_mosaicism != -1:
            # there is a t_value that exceeds the thresold, save the vcf position of the start
            vcf_pos_start_of_mosaicism = vcf_pos[index_of_mosaicism + samp_size -1] if index_of_mosaicism != 0 else vcf_pos[0]
            # figure out the end point of mosaicism
            index_of_end_of_mosaicism = next((i+index_of_mosaicism+1 for i, elem in enumerate(t_values[index_of_mosaicism + 1:]) if elem < t_thres),len(t_values) - 1)
            vcf_pos_end_of_mosaicism = vcf_pos[index_of_end_of_mosaicism + samp_size - 1] if index_of_end_of_mosaicism != len(t_values) - 1 else vcf_pos[-1]
            return [vcf_pos_start_of_mosaicism, vcf_pos_end_of_mosaicism]
        else:
            return None

    
def runner(child, chr_name, sample_size, t_threshold):
    # step 1: filter the SNP df by chromosome name
    chr_snp_df = helper_functions.chromosome_filter(child.SNP_df, chr_name)
    # step 2: do phasing and return results
    vcf_pos, maternal_rd, paternal_rd = child.phasable_snp_determiner(chr_snp_df)
    results = sandia_t_test_snps(vcf_pos, maternal_rd, paternal_rd, samp_size=sample_size, t_thres=t_threshold)
    return results

In [13]:
# step 6: Generate results
start = time.time()
mosaicism_outcome = [runner(proband, chr_name, int(config_elem["SAMPLE_SIZE"]), float(config_elem["T_THRES"])) for chr_name in chromosome_list]
end = time.time()
print(str((end-start)/60) + " minutes")

8.118711630503336 minutes


In [14]:
# step 7: write results to file
with open(config_elem["OUTPUT_FILE"], 'w') as output_file:
    for i, elem in enumerate(mosaicism_outcome):
        if elem is not None:
            line = ["There is mosaicism in chromosome", str(chromosome_list[i]), "starting at VCF position", str(elem[0]), "and ending at VCF position", str(elem[1])]
            output_file.write(" ".join(line))
            output_file.write("\n")

In [ ]:
# Verify results 